# Installing all the needed packages (Run this only if you're using the Google Collab)

In [3]:
# apt-get handle the installation and removal of software on Debian, Ubuntu, and related Linux distributions
!apt-get update

# installing needed packages
!apt install chromium-chromedriver
!pip install selenium
!pip install beautifulsoup4

# Copy the chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
# This make sure that the selenium chromedriver is in the path
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease    
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease    
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.n

# Settings all imports

In [4]:
from google.colab import drive
#drive.mount("/content/drive")
drive.mount("/content/ShardDrives")

import tqdm
import requests
import urllib.request

import numpy as np
import pandas as pd

from datetime import date, timedelta
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from bs4.element import Comment
# from google.colab import drive

Drive already mounted at /content/ShardDrives; to attempt to forcibly remount, call drive.mount("/content/ShardDrives", force_remount=True).


# Create a web driver based on operating system. Please change the value of the `OPERATING_SYSTEM` constant to run this notebook on your system

In [5]:
OPERATING_SYSTEM = "linux"

if OPERATING_SYSTEM == "windows":
    executable_path = "drivers/windows/chromedriver.exe"
elif OPERATING_SYSTEM == "mac":
    executable_path = "drivers/mac/chromedriver"
elif OPERATING_SYSTEM == "linux":
    executable_path = "chromedriver"

base_url = 'https://www.jobs.bg/'

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(executable_path=executable_path, options=chrome_options)


"""url = 'https://www.jobs.bg/front_job_search.php?frompage=0&add_sh=1&from_hp=1&term=#paging'
r = driver.get(url)
html = driver.execute_script("return document.documentElement.outerHTML")
soup = BeautifulSoup(html, 'html.parser')"""

'url = \'https://www.jobs.bg/front_job_search.php?frompage=0&add_sh=1&from_hp=1&term=#paging\'\nr = driver.get(url)\nhtml = driver.execute_script("return document.documentElement.outerHTML")\nsoup = BeautifulSoup(html, \'html.parser\')'

# Extracting the category names and ids so we can perform filters based on category

In [6]:
def extract_categories_dataframe():
    """"""
    url = "https://www.jobs.bg/index.php"
    r = driver.get(url)
    categories_button = driver.find_element_by_id("categoriesChip")
    categories_button.click()
    html = driver.execute_script("return document.documentElement.outerHTML")
    soup = BeautifulSoup(html, 'html.parser')

    categories_elem = soup.find('div',attrs={'id':'categoriesSelectSheet'})
    categories_elements = categories_elem.find_all("div", attrs={"class": "mdc-chip mdc-ripple-upgraded"})

    categories_info = []
    categories_info.append((56))

    for category_element in categories_elements:
        category_id = category_element['id'].split("_")[-1]
        category_id = int(category_id)
        categories_info.append(category_id)

    #categories_info = pd.DataFrame(categories_info, columns=["category_id"])

    return categories_info

In [7]:
cats_selected = [25, 1]
cats_selected

[25, 1]

In [8]:
def get_position_id(text_info_elem):
    """
    """
    class_id_elem = 'card__title mdc-typography mdc-typography--headline6 text-overflow'
    id_elem = text_info_elem.find('a',attrs={'class':class_id_elem})['href']
    position_id = id_elem.lstrip('job/')
    
    return position_id

def extract_job_position_id_as_array(base_url, text_info_elem): # , job_company_elem):
    """
    """
    position_id = get_position_id(text_info_elem)

    current_job_info = [position_id]
    
    return current_job_info

In [9]:
def extract_jobs(start_page, end_page):
    """"""
    position_category_list = []

    for category in tqdm.tqdm(cats_selected):
        for page in tqdm.tqdm(range(start_page, end_page)):
    
            try:
                # This is the base url looping through all categories and specified pages in order
                url = "https://www.jobs.bg/front_job_search.php?frompage={}&add_sh=1&categories%5B0%5D={}&term=#paging".format(page*15, category)
              
                r = driver.get(url)
                html = driver.execute_script("return document.documentElement.outerHTML")
                soup = BeautifulSoup(html, 'html.parser')
                
                class_id_elem = 'card__title mdc-typography mdc-typography--headline6 text-overflow'
                id_elem = soup.find_all('a',attrs={'class':class_id_elem})
            except Exception as e:
                print(f"Page number: {page}, url: {url}")
                print(f"Unexpected error: {e.message}")

            for i in range(0,15,1):
                try:
                    position_id = id_elem[i]['href'].lstrip('job/')
                    position_id = (position_id, category)
                    position_category_list.append(position_id)

                except Exception as e:
                    print(f"Curent page: {page}, {category}, {position_id}")
                    print(f"Unexpected error: {e}")

    df_columns = ["position_id", "category"]

    position_categofy_df = pd.DataFrame(position_category_list, columns=df_columns)

    return position_categofy_df

In [10]:
jobs = extract_jobs(0,10)

 70%|███████   | 7/10 [00:08<00:03,  1.17s/it]

Curent page: 6, 25, ('5691208', 25)
Unexpected error: list index out of range
Curent page: 6, 25, ('5691208', 25)
Unexpected error: list index out of range
Curent page: 6, 25, ('5691208', 25)
Unexpected error: list index out of range
Curent page: 6, 25, ('5691208', 25)
Unexpected error: list index out of range
Curent page: 6, 25, ('5691208', 25)
Unexpected error: list index out of range
Curent page: 6, 25, ('5691208', 25)
Unexpected error: list index out of range
Curent page: 6, 25, ('5691208', 25)
Unexpected error: list index out of range
Curent page: 6, 25, ('5691208', 25)
Unexpected error: list index out of range
Curent page: 6, 25, ('5691208', 25)
Unexpected error: list index out of range
Curent page: 6, 25, ('5691208', 25)
Unexpected error: list index out of range
Curent page: 6, 25, ('5691208', 25)
Unexpected error: list index out of range
Curent page: 6, 25, ('5691208', 25)
Unexpected error: list index out of range
Curent page: 6, 25, ('5691208', 25)
Unexpected error: list index


 80%|████████  | 8/10 [00:09<00:02,  1.05s/it]

Curent page: 7, 25, ('5691208', 25)
Unexpected error: list index out of range
Curent page: 7, 25, ('5691208', 25)
Unexpected error: list index out of range
Curent page: 7, 25, ('5691208', 25)
Unexpected error: list index out of range
Curent page: 7, 25, ('5691208', 25)
Unexpected error: list index out of range
Curent page: 7, 25, ('5691208', 25)
Unexpected error: list index out of range
Curent page: 7, 25, ('5691208', 25)
Unexpected error: list index out of range
Curent page: 7, 25, ('5691208', 25)
Unexpected error: list index out of range
Curent page: 7, 25, ('5691208', 25)
Unexpected error: list index out of range
Curent page: 7, 25, ('5691208', 25)
Unexpected error: list index out of range
Curent page: 7, 25, ('5691208', 25)
Unexpected error: list index out of range
Curent page: 7, 25, ('5691208', 25)
Unexpected error: list index out of range
Curent page: 7, 25, ('5691208', 25)
Unexpected error: list index out of range
Curent page: 7, 25, ('5691208', 25)
Unexpected error: list index


 90%|█████████ | 9/10 [00:10<00:00,  1.02it/s]

...

Curent page: 7, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page: 7, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page: 7, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page: 7, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page: 7, 1, ('5691986', 1)
Unexpected error: list index out of range



 90%|█████████ | 9/10 [00:08<00:00,  1.05it/s]

Curent page: 8, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page: 8, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page: 8, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page: 8, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page: 8, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page: 8, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page: 8, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page: 8, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page: 8, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page: 8, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page: 8, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page: 8, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page: 8, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page:


100%|██████████| 2/2 [00:21<00:00, 10.56s/it]

Curent page: 9, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page: 9, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page: 9, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page: 9, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page: 9, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page: 9, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page: 9, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page: 9, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page: 9, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page: 9, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page: 9, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page: 9, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page: 9, 1, ('5691986', 1)
Unexpected error: list index out of range
Curent page:

In [11]:
jobs

,position_id,category
0,5726080,25
1,5720106,25
2,5710686,25
3,5726320,25
4,5718103,25
...,...,...
200,5694610,1
201,5693502,1
202,5693523,1
203,5692026,1


In [12]:
jobs.to_csv("/content/ShardDrives/MyDrive/jobsbg_scrapped/categories_20210610.csv", index=False)